In this file, I am using hugging face embedding model and llm model. 
Both are free but the llm model seems not to be very strong.


In [1]:
import getpass
import os
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

# Set Hugging Face API key
if not os.environ.get("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter API key for Hugging Face: ")

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model= "mixedbread-ai/mxbai-embed-large-v1",
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
)


In [2]:
# from langchain_huggingface import HuggingFaceEmbeddings

# embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-mpnet-base-v2",
# )

In [8]:
texts = ["Hello, world!", "How are you?"]
embedding_vector = hf_embeddings.embed_documents(texts)
print(embedding_vector[:5])  # Print first 5 dimensions for preview

[[0.30596935749053955, 0.7907294034957886, 0.009807621128857136, -0.15764321386814117, -0.8023926019668579, 0.14975742995738983, 1.2789502143859863, 0.7509116530418396, 0.8415459990501404, 0.042983416467905045, 0.12561291456222534, 0.0820784866809845, -0.09308472275733948, 0.1274476796388626, -0.8875035047531128, -0.046241745352745056, -0.16150838136672974, -0.682961106300354, -0.7650361657142639, 0.03389454632997513, -0.3529984951019287, 0.3765198290348053, -1.2507705688476562, -0.3606584370136261, 0.21749347448349, 0.3550860285758972, 0.8079591393470764, -0.07037495076656342, 0.8502594828605652, 0.851040780544281, -0.2966594696044922, 0.16541153192520142, -0.2534833550453186, -1.0056427717208862, -0.46689340472221375, -0.18940310180187225, 0.40086567401885986, -0.3046886920928955, -0.5170363187789917, -0.644966721534729, 0.5159122347831726, 0.22100073099136353, 0.9491286873817444, -0.5215340256690979, -1.3068257570266724, 0.35012808442115784, 0.03954598307609558, 0.23762449622154236,

In [3]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.schema import Document

# Initialize the in-memory vector store
vector_store2 = InMemoryVectorStore(hf_embeddings)

# Example paragraph
paragraph = "Face is the front part of a person's head, including eyes, nose, and mouth. Flower is a living creature, including petals, stem, and roots. Head is the upper part of the human body, containing the brain, eyes, ears, nose, and mouth."

# Convert paragraph into a document object
doc = Document(page_content=paragraph)

# Add the document to the vector store
vector_store2.add_documents([doc])

# Test retrieval with a query
query = "What is a flower?"
retrieved_docs = vector_store2.similarity_search(query, k=1)

# Print retrieved document
print("Retrieved:", retrieved_docs[0].page_content)

c:\Users\LEEH007\source\AI\First_RAG\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Retrieved: Face is the front part of a person's head, including eyes, nose, and mouth. Flower is a living creature, including petals, stem, and roots. Head is the upper part of the human body, containing the brain, eyes, ears, nose, and mouth.


In [4]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-Nemo-Instruct-2407",  # Change to your preferred model
    model_kwargs={"temperature": 0.1, "max_length": 50},
)

C:\Users\LEEH007\AppData\Local\Temp\ipykernel_16556\2993362245.py:3: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [5]:
# Retrieve most relevant document
retrieved_text = retrieved_docs[0].page_content

# Summarize it to 15 words
# Explicit summarization prompt
summary_prompt = (
    # "Task: Summarize the following text **in exactly 15 words**.\n\n"
    # "Text:\n"
    # f"{retrieved_text}\n\n"
    # "Summary (15 words only):"
    f"""
    Use the given context to answer user's query.

    Query:
    {query}

    Context:
    {retrieved_text} 
    
    Give a concise answer in one sentence.

    """.strip()
)
short_summary = llm.invoke(summary_prompt)

print("Short Output:", short_summary)



c:\Users\LEEH007\source\AI\First_RAG\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Short Output: Use the given context to answer user's query.

    Query:
    What is a flower?

    Context:
    Face is the front part of a person's head, including eyes, nose, and mouth. Flower is a living creature, including petals, stem, and roots. Head is the upper part of the human body, containing the brain, eyes, ears, nose, and mouth. 
    
    Give a concise answer in one sentence.


In [6]:

summary_prompt = (
    "Task: What are the entities?**.\n\n"
    "Text:\n"
    f"{retrieved_text}\n\n"
    ":"
)
short_summary = llm.invoke(summary_prompt)

print("another Output:", short_summary)

c:\Users\LEEH007\source\AI\First_RAG\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


another Output: Task: What are the entities?**.

Text:
Face is the front part of a person's head, including eyes, nose, and mouth. Flower is a living creature, including petals, stem, and roots. Head is the upper part of the human body, containing the brain, eyes, ears, nose, and mouth.

: Entities:
- Face
- Flower
- Head


In [7]:
# Test the LLM with a Prompt
response = llm.invoke("What is hugging face?")
print(response)


c:\Users\LEEH007\source\AI\First_RAG\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


What is hugging face? Hugging Face is a technology company based in New York and Paris that specializes in natural language processing (NLP). They are known for their popular open-source transformer models, such as BERT, RoBERTa, and T5, which are used for a wide range of NLP tasks, including text classification, question answering, and language translation. Hugging Face also provides a platform for developers to share and collaborate on NLP models, as well as a library of pre-trained models that can be fine-tuned for specific use cases. Additionally, they offer a range of tools and services for NLP, including a model hub, a dataset hub, and a transformers library.
